In [1]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper

    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses

    scraping.contract_addresses.main()

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.
    import gbq.gbq

    gbq.gbq.main()


main()


2023-04-03 16:32:11,470 - INFO - Cloud function starting.
2023-04-03 16:32:12,691 - INFO - Scraping complete.
2023-04-03 16:32:13,463 - INFO - Scraping 3 coins that are missing symbols.
2023-04-03 16:32:17,207 - INFO - Scraped https://www.coingecko.com/en/coins/sxp
2023-04-03 16:32:20,913 - INFO - Scraped https://www.coingecko.com/en/coins/injective
2023-04-03 16:32:24,744 - INFO - Scraped https://www.coingecko.com/en/coins/panther-protocol
2023-04-03 16:32:24,770 - INFO - Number of coins with ethereum contract addresses: 19
2023-04-03 16:32:24,771 - INFO - Done scraping contract addresses!
2023-04-03 16:32:24,775 - INFO - Checking 19 token contracts.
2023-04-03 16:32:24,960 - INFO - Running SQL query on Google Big Cloud.
2023-04-03 16:32:28,313 - INFO - Query will process 768 MB.
2023-04-03 16:32:28,316 - INFO - Running query...
2023-04-03 16:32:29,896 - INFO - Query completed.
2023-04-03 16:32:36,537 - INFO - Found 31 wallets.


In [1]:
import gbq.gbq

gbq.gbq.main(True)

2023-04-03 16:36:03,966 - INFO - Checking 19 token contracts.



    WITH wallet_token_counts AS (
    SELECT tt.to_address, tt.token_address, COUNT(tt.to_address) as num_txns
    FROM `bigquery-public-data.crypto_ethereum.token_transfers` tt 
    WHERE tt.block_timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) 
    AND tt.block_timestamp < TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 2 DAY) 
    AND tt.token_address in ('0x2c5bc2ba3614fd27fcc7022ea71d9172e2632c16','0x7b4328c127b85369d9f82ca0503b000d09cf9180','0x97872eafd79940c7b24f7bcc1eadb1457347adc9','0xf3b9569f82b18aef890de263b84189bd33ebe452','0x5de8ab7e27f6e7a1fff3e5b337584aa43961beef','0xb1f871ae9462f1b2c6826e88a7827e76f86751d4','0x177d39ac676ed1c67a2b268ad7f1e58826e5b0af','0xfcf8eda095e37a41e002e266daad7efc1579bc0a','0x96610186f3ab8d73ebee1cf950c750f3b1fb79c2','0x06450dee7fd2fb8e39061434babcfc05599a6fb8','0x3e9bc21c9b189c09df3ef1b824798658d5011937','0x505b5eda5e25a67e1c24a2bf1a527ed9eb88bf04','0xed04915c23f00a313a544955524eb7dbd823143d','0x73968b9a57c6e53d41345fd57a6e6ae27d6cdb2